# Sorghum Cultivar Identification
## Training for convolution neural networks
* Veronica Thompson
* Colorado State University Global
* MIS 581: Capstone
* Dr. Orenthio Goodwin
* 5/15/2022

### import libraries

In [ ]:
from matplotlib import pyplot
from os import listdir
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, Dropout, MaxPooling2D, Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import SGD
import tensorflow as tf

### import proper preprocess_input for vgg16 or resnet50 models

In [ ]:
# from keras.applications.vgg16 import preprocess_input
# from tensorflow.keras.applications.resnet50 import preprocess_input

### Choose 224x224 or 128x128 pixels
* Full set of input images was scaled to both 224x224 and 128x128 pixels
* Show sample images

In [ ]:
size = 128
# size = 224

In [ ]:
if size == 224:
    train_folder = '../input/sorghum-224x224/train'
else:
    train_folder = '../input/sorghum-cultivar-identification-128128/train'

In [ ]:
for i, filename in zip(range(9), listdir(train_folder)[:9]):
    pyplot.subplot(330 + 1 + i)
    image = pyplot.imread(train_folder + '/' + filename)
    pyplot.imshow(image)
pyplot.show()

### mapping_csv contains list of training image files and their cultivar classification

In [ ]:
mapping_csv = pd.read_csv('sorghum-id-fgvc-9/train_cultivar_mapping.csv')
mapping_csv.dropna(inplace=True)
mapping_csv.head()

In [ ]:
### look for missing image files

list_f = listdir(train_folder)
for index, data in mapping_csv.iterrows():
    if not data['image'] in list_f:
        print(data['image'])

## Baseline Model
Baseline model has one VGG-style block with two convolution laters and a max pooling layer. Model definition function also allows two or three VGG-style blocks and optional dropout.

In [ ]:
def define_model_baseline(shape=(128,128,3), layers=1, dropout=False):
    model = Sequential()
    model.add(Conv2D(32, (3,3), activation='relu', 
                     kernel_initializer='he_uniform',
                     padding='same', input_shape=shape))
    model.add(Conv2D(32, (3,3), activation='relu', 
             kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    if dropout:
        model.add(Dropout(0.2))
    if layers >= 2:
        model.add(Conv2D(64, (3,3), activation='relu', 
                         kernel_initializer='he_uniform', padding='same'))
        model.add(Conv2D(64, (3,3), activation='relu', 
                         kernel_initializer='he_uniform', padding='same'))
        model.add(MaxPooling2D((2, 2)))
        if dropout:
            model.add(Dropout(0.2))
    if layers >= 3:
        model.add(Conv2D(128, (3,3), activation='relu', 
                         kernel_initializer='he_uniform', padding='same'))
        model.add(Conv2D(128, (3,3), activation='relu', 
                         kernel_initializer='he_uniform', padding='same'))
        model.add(MaxPooling2D((2, 2)))
        if dropout:
            model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    if dropout:
        model.add(Dropout(0.5))
    model.add(Dense(100, activation='softmax'))
    opt = SGD(learning_rate=0.01, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    return model

## VGG16
Pretrained VGG16 model with custom classification layer.

In [ ]:
def define_model_vgg16(shape=(128, 128, 3)):
    model = VGG16(include_top=False, input_shape=shape)
    for layer in model.layers:
        layer.trainable = False
              
    flat1 = Flatten()(model.layers[-1].output)
    class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
    output = Dense(100, activation='softmax')(class1)
    model = Model(inputs=model.inputs, outputs=output)
    opt = SGD(learning_rate=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    return model

## ResNet50
Pretrained ResNet50 model with custom classification layer.

In [ ]:
def define_model_resnet50(shape=(128, 128, 3)):
    model = ResNet50(include_top=False, input_shape=shape)
    model.trainable = False
              
    flat1 = Flatten()(model.layers[-1].output)
    class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
    output = Dense(100, activation='softmax')(class1)
    model = Model(inputs=model.inputs, outputs=output)
    opt = SGD(learning_rate=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    return model

## Training evaluation
### Display loss and accuracy plots, save training histories

In [ ]:
def summarize_diagnostics(history):
    pyplot.subplot(211)
    pyplot.title('Cross Entropy Loss')
    pyplot.plot(history.history['loss'], color='blue', label='train')
    pyplot.plot(history.history['val_loss'], color='orange', label='test')
    pyplot.subplot(212)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['accuracy'], color='blue', label='train')
    pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
    pyplot.show()

In [ ]:
def save_diagnostics(history, filename):
    df = pd.DataFrame(history.history)
    with open(filename, mode='w') as f:
        df.to_csv(f)

### Train Model
Trains selected model, saves model to .h5 file and training history to csv file

In [ ]:
def train_from_data_frame(saved_model=None):
    
    # Use rescale for baseline model, preprocessing function for VGG16 and ResNet50
    
    datagen_train = ImageDataGenerator(rescale=1.0/255.0,
                                       # width_shift_range=0.1,
                                       # height_shift_range=0.1,
                                       # horizontal_flip=True,
                                       # vertical_flip=True,
                                       # preprocessing_function=preprocess_input)
    
    datagen_test = ImageDataGenerator(rescale=1.0/255.0
                                      # preprocessing_function=preprocess_input)
    
    # divide training data into train and validation sets
    train_df, test_df = train_test_split(mapping_csv, test_size=0.2, random_state=1)
    
    #prepare iterators
    train_it = datagen_train.flow_from_dataframe(dataframe=train_df,
                                                 directory=train_folder,
                                                 x_col='image',
                                                 y_col='cultivar',
                                                 class_mode='categorical',
                                                 target_size=(size, size),
                                                 batch_size=128)
    test_it = datagen_test.flow_from_dataframe(dataframe=test_df,
                                               directory=train_folder,
                                               x_col='image',
                                               y_col='cultivar',
                                               class_mode='categorical',
                                               target_size=(size, size),
                                               batch_size=128)

    if saved_model:
        model = load_model(saved_model)
    else:       
        model = define_model_baseline(shape=(size,size,3), layers=1, dropout=False)
        # model = define_model_vgg16(shape=(size,size,3))
        # model = define_model_resnet50(shape=(size,size,3))
    
    # print(model.summary())
    
    history = model.fit(train_it,
                        steps_per_epoch=len(train_it),
                        validation_data=test_it,
                        validation_steps=len(test_it),
                        epochs=10)
    _, acc = model.evaluate(test_it, steps=len(test_it))
    # display accuracy results
    print('> %.3f' % (acc * 100.0))
    summarize_diagnostics(history)
                                      
    # save model and training history                                  
    out_name = 'baseline_128x128'
    save_diagnostics(history, out_name + '_history.csv')
    model.save(out_name + '_model.h5')

In [ ]:
train_from_data_frame()

In [ ]:
# train_from_data_frame('resnet50_shiftflip_ep111-130_model.h5')